In [10]:
#importing the required libraries
import numpy as np
import torch
import torchvision
import matplotlib.pyplot as plt
from time import time
from torchvision import datasets, transforms
from torch import nn, optim

In [11]:
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,), (0.5,)),])

In [12]:
#To get the image
trainset = datasets.MNIST('PATH_TO_STORE_TRAINSET', download=True, train=True, transform=transform)
valset = datasets.MNIST('PATH_TO_STORE_TESTSET', download=True, train=False, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
valloader = torch.utils.data.DataLoader(valset, batch_size=64, shuffle=True)

In [13]:
dataiter = iter(trainloader)
images, labels = next(dataiter)

In [14]:
input_size = 784
hidden_sizes = [128, 64]
output_size = 10

model = nn.Sequential(nn.Linear(input_size, hidden_sizes[0]),
                      nn.ReLU(),
                      nn.Linear(hidden_sizes[0], hidden_sizes[1]),
                      nn.ReLU(),
                      nn.Linear(hidden_sizes[1], output_size),
                      nn.LogSoftmax(dim=1))
print(model)

Sequential(
  (0): Linear(in_features=784, out_features=128, bias=True)
  (1): ReLU()
  (2): Linear(in_features=128, out_features=64, bias=True)
  (3): ReLU()
  (4): Linear(in_features=64, out_features=10, bias=True)
  (5): LogSoftmax(dim=1)
)


In [15]:
criterion = nn.NLLLoss()
images, labels = next(iter(trainloader))
images = images.view(images.shape[0], -1)

logps = model(images) #log probabilities
loss = criterion(logps, labels) #calculate the NLL loss

In [16]:
print('Before backward pass: \n', model[0].weight.grad)
loss.backward()
print('After backward pass: \n', model[0].weight.grad)

Before backward pass: 
 None
After backward pass: 
 tensor([[-1.8455e-04, -1.8455e-04, -1.8455e-04,  ..., -1.8455e-04,
         -1.8455e-04, -1.8455e-04],
        [ 3.8295e-03,  3.8295e-03,  3.8295e-03,  ...,  3.8295e-03,
          3.8295e-03,  3.8295e-03],
        [-3.5904e-03, -3.5904e-03, -3.5904e-03,  ..., -3.5904e-03,
         -3.5904e-03, -3.5904e-03],
        ...,
        [ 3.4706e-03,  3.4706e-03,  3.4706e-03,  ...,  3.4706e-03,
          3.4706e-03,  3.4706e-03],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [-2.5252e-06, -2.5252e-06, -2.5252e-06,  ..., -2.5252e-06,
         -2.5252e-06, -2.5252e-06]])


In [17]:
optimizer = optim.SGD(model.parameters(), lr=0.003, momentum=0.9)
time0 = time()
epochs = 15
for e in range(epochs):
    running_loss = 0
    for images, labels in trainloader:
        # Flatten MNIST images into a 784 long vector
        images = images.view(images.shape[0], -1)
    
        # Training pass
        optimizer.zero_grad()
        
        output = model(images)
        loss = criterion(output, labels)
        
        #This is where the model learns by backpropagating
        loss.backward()
        
        #And optimizes its weights here
        optimizer.step()
        
        running_loss += loss.item()
    else:
        print("Epoch {} - Training loss: {}".format(e, running_loss/len(trainloader)))
print("\nTraining Time (in minutes) =",(time()-time0)/60)

Epoch 0 - Training loss: 0.6122874769765431
Epoch 1 - Training loss: 0.28365842936072017
Epoch 2 - Training loss: 0.2282653022716358
Epoch 3 - Training loss: 0.18436929540657032
Epoch 4 - Training loss: 0.15480086630158651
Epoch 5 - Training loss: 0.13047882272149008
Epoch 6 - Training loss: 0.11337113403566261
Epoch 7 - Training loss: 0.09965089589023768
Epoch 8 - Training loss: 0.0890775175729413
Epoch 9 - Training loss: 0.0806968087878531
Epoch 10 - Training loss: 0.07365913857231295
Epoch 11 - Training loss: 0.06688083932776727
Epoch 12 - Training loss: 0.06238644443569518
Epoch 13 - Training loss: 0.05790874802533275
Epoch 14 - Training loss: 0.0523542633790499

Training Time (in minutes) = 2.3365355253219606


In [18]:
images, labels = next(iter(valloader))

img = images[0].view(1, 784)
with torch.no_grad():
    logps = model(img)

ps = torch.exp(logps)
probab = list(ps.numpy()[0])
print("Predicted Digit =", probab.index(max(probab)))


Predicted Digit = 0


In [21]:
correct_count, all_count = 0, 0
for images,labels in valloader:
  for i in range(len(labels)):
    img = images[i].view(1, 784)
    with torch.no_grad():
        logps = model(img)

    
    ps = torch.exp(logps)
    probab = list(ps.numpy()[0])
    pred_label = probab.index(max(probab))
    true_label = labels.numpy()[i]
    if(true_label == pred_label):
      correct_count += 1
    all_count += 1

print("Number Of Images Tested =", all_count)
print("\n")
print("Model Accuracy =", (correct_count/all_count))

Number Of Images Tested = 10000


Model Accuracy = 0.9729


In [20]:
torch.save(model, './my_mnist_model.pt')